# Shaft Speed Selection and Analysis Document

> Author: Elias Aoubala

> Date: 27/03/2025

## 1 - Background

This document contains the analysis and justification for the selection of the preliminary selection of the shaft speed of the turbopump.
A basic literature review is conducted to see what has been done, what is the current state-of-the-art, and what are the key drivers for the selection of the shaft speed of a turbopump.

## 2 - Literature Review

The shaft speed of turbopumps is an extremely influential parameter when it comes to the design to turbopump-driven cycles, as often it will drive the expected cycle performances of the system, coupled with being the key-contributor to the expected stresses the mechanical assembly will see.

Before discussing in detail the impacts of shaft speed selection have on the overall subsystem, for the sake of brevity, **discussion will be limited to assemblies which have a single shaft, just like the intended architecture of the Mermaid-man turbopump assembly, along with gas-generator based engine cycles.**

In-order to get a better idea of what has been done in terms of shaft speeds, a brief litterature review has been conducted to identify what has typically been used succesfully in the past on engines with significant heritage.

### 2.1 - Heritage Designs

A table of the engine name, propellants and turbopump shaft speeds have been presented below.

| Organsiation | Engine/Turbopump Name | Propellants | Shaft Speed (RPM) | Citation                                                                          |
| ------------ | --------------------- | ----------- | ----------------- | --------------------------------------------------------------------------------- |
| SpaceX       | Merlin 1-A            | LOX/RP-1    | 20,000            |                                                                                   |
| SpaceX       | Merlin 1-B            | LOX/RP-1    | 22,000            |                                                                                   |
| SpaceX       | Merlin 1-D            | LOX/RP-1    | 36,000            |                                                                                   |
| Perigee      | Blue 1-S              | LOX/LCH4    | 50,000            |                                                                                   |
| NASA         | Fastrac               | LOX/RP-1    | 20,000            | [NTRS](https://ntrs.nasa.gov/api/citations/19990009133/downloads/19990009133.pdf) |
| Ariane       | Vulcain 2             | LOX         | 35,680            | [Trollheden et al.](https://doi.org/10.2514/6.1999-2342)                          |
| Ariane       | Vulcain 2             | LH2         | 13,290            | [Mark and Windstrom](https://arc.aiaa.org/doi/abs/10.2514/6.1995-2404)            |
|              |                       |             |                   |                                                                                   |
